In [8]:
import pandas as pd
from charactertraining.constants import DATA_PATH
from charactertraining.questions import TRAITS
from charactertraining.claude_questions import CLAUDE_TRAITS

In [9]:
questions = pd.DataFrame(columns=["trait", "question"])
questions_large = pd.DataFrame(columns=["trait", "question"])
for trait in TRAITS:
    questions.loc[len(questions)] = [trait, TRAITS[trait]]
    questions_large.loc[len(questions_large)] = [trait, TRAITS[trait] + CLAUDE_TRAITS[trait]]
questions.to_json(f"{DATA_PATH}/critiques/inputs/questions.jsonl", orient="records", lines=True)
questions_large.to_json(f"{DATA_PATH}/critiques/inputs/questions_large.jsonl", orient="records", lines=True)